# Functionality: visualization of functional connectivity

<ul>
    <li>142 channels = 1-96 channels (M1) + 97-100(not used) + 101-132(ThaSMA) + 133-146 (GP/STN)</li>
    <li>load animetafolder/pinky_locThaSMA.csv for the details of ThaSMA </li>
</ul>

## Import Packages 

In [1]:
import os, sys
import numpy as np
import pandas as pd

import pickle

from igraph.drawing.text import TextDrawer
import cairo

In [2]:
# import_nbmodule used for import package in .ipynb
import import_nbmodule

# extract the exp folder path
currfolder = os.getcwd()
codefolder = currfolder[0 : currfolder.find('code')+len('code')]

# add path the exp folder
sys.path.append(codefolder)


# import animal_folder function in util/folder_extract.pynb 
from util.folder_extract import exp_subfolders, code_corresfolder

# import graphVisual in connAnalyTool/graphVisual.pynb 
from connAnalyTool import graphVisual

importing Jupyter notebook from /Users/linglingyang/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/util/folder_extract.ipynb
importing Jupyter notebook from /Users/linglingyang/yang7003@umn.edu/NMRC_umn/Projects/FCAnalysis/exp/code/connAnalyTool/graphVisual.ipynb


## exp subfolders & code_corresfolder

In [3]:
%%javascript
IPython.notebook.kernel.execute('nb_name = "' + IPython.notebook.notebook_name + '"')

<IPython.core.display.Javascript object>

In [4]:
nb_name = nb_name[0: nb_name.find('.ipynb')]

# corresfolder
corresfolder = code_corresfolder(os.getcwd(), nb_name)

## global parameters

In [ ]:
# threshold lowweight
lowweight = 0.1

# frequency
freq = [17, 19]
strfreq = 'freq' + str(freq[0]) + '_' + str(freq[1])


# eventphase
eventphase = 'return'

In [ ]:
### text setup for brain areas ###
texts_area = dict()

texts_area['VPLo'] = [20, 40, 20]

texts_area['VLo'] = [20, 120, 20]

texts_area['VA'] = [20, 200, 20]

texts_area['SMA'] = [20, 320, 20]

texts_area['M1'] = [20, 500, 20]

texts_area['STN'] = [280, 140, 20]

texts_area['GP'] = [360, 250, 20]

In [ ]:
areas_unwanted = ['lCd','rMC']

## input setup

In [ ]:
inputfolder = os.path.join(correparentfolder, 'm1_calcciCOH_rest')
filename = 'ciCOH_rest_freq10_12.pickle'

file = os.path.join(inputfolder, filename)

## savefile setup

In [ ]:
savefolder = corresfolder
savefile_prefix = 'ciCOH_' + eventphase + '_' + strfreq + '_lowweight' + str(lowweight)

## extract ciCOH

In [ ]:
# load dictionary ciCOH
dic_ciCOH = pickle.load(open(ciCOHfile, "rb" ))

# ciCOH_normal, ciCOH_mild: np.ndarray, shape = (nchns, nchns, ntrials)
ciCOH_normal, ciCOH_mild = dic_ciCOH['ciCOH_normal'], dic_ciCOH['ciCOH_mild']

## ciCOH Visual

In [ ]:
def ciCOH_visual_save(ciCOH, chn_area, chn_coord, str_recchnishow, lowweight, tobesavedfile, texts):
    """
        
        Args:
            
            ciCOH (np.sdarray): ciCOH Tensor for each trial (nchns, nchns, ntrials)
            
            chn_area (list): a list of string representing brain area for each channel, len(chn_area) = nchns
            
            chn_coord (numpy.ndarray): x, y coordinates for each channel, shape = (nchns, 2)
            
            chn_recchni(list): showed recording chni for each channel, shape = (nchns, )
            
            lowweight: the threshold lowweight, only weight>lowweight is treated as connection
            
            tobesavedfile: file to save the visualized figure
            
            texts:
            
        Output:
            the visualizaton of ciCOH is saved in tobesavedfile
            
    """
    
    weight = abs(np.mean(ciCOH, axis = 2))

    # weight > lowweight
    weight[np.where(weight<lowweight)] =0
    
    graph = graphVisual.graph_create(weight, vsarea = chn_area, vscoord = chn_coord, vschni = str_recchnishow)

    # set graph visualization style
    visual_style = graphVisual.graph_style(graph)
    

    igplot = graphVisual.graph_plot(graph, visual_style, texts = texts)
    
    igplot.save(tobesavedfile)
    
    return igplot

### outer 

In [ ]:
str_recchnishow = ['']*96 + [str(i) for i in chnInf_recchni[96:]]
print(type(str_recchnishow))

In [ ]:
str_recchnishow = ['']*123

In [ ]:
"""
    normal condition
"""

# extract the first 128 channels for M1 and ThaSMA areas
ciCOH_normal_subchns = ciCOH_normal[0:128, 0:128, :]


# delete the unused channels in weight
ciCOH_normal_subchns = np.delete(ciCOH_normal_subchns, idx_notused, axis=0)
ciCOH_normal_subchns = np.delete(ciCOH_normal_subchns, idx_notused, axis=1)


# save file config
savefilename = savefile_prefix + '_normal' + '.png'
savefile = os.path.join(savefolder, savefilename)

# printed text config
texts = texts_area.copy()
title = 'normal ' + strfreq 
texts[title] = pos_title



###### --- change texts #########
del texts['reach'], texts['normal freq9_11'], texts['Caudate'], texts['VPLo'], texts['VLo'], texts['VA'], texts['MC'], texts['SMA'], texts['M1']





# visualize and save
igplot = ciCOH_visual_save(ciCOH_normal_subchns, 
                           chn_area = chnInf_brainarea, 
                           chn_coord = chnInf_coord,
                           str_recchnishow = str_recchnishow,
                           lowweight = lowweight,  
                           tobesavedfile = savefile, 
                           texts = texts)

igplot.show()

print("Figure saved to " + savefile)


# del 
#del ciCOH_normal_subchns, savefile, texts, title

In [ ]:
"""
    mild condition
"""

condition = 'mild'

# extract the first 128 channels for M1 and ThaSMA areas
ciCOH_mild_subchns = ciCOH_mild[0:128, 0:128, :]


# delete the unused channels in weight
ciCOH_mild_subchns = np.delete(ciCOH_mild_subchns, idx_notused, axis=0)
ciCOH_mild_subchns = np.delete(ciCOH_mild_subchns, idx_notused, axis=1)


# save file config
savefilename = savefile_prefix + '_' + condition + '.png'
savefile = os.path.join(savefolder, savefilename)

# printed text config
texts = texts_area.copy()
title =  condition + ' ' + strfreq 
texts[title] = pos_title


###### --- change texts #########
del texts['reach'], texts['mild freq9_11'], texts['Caudate'], texts['VPLo'], texts['VLo'], texts['VA'], texts['MC'], texts['SMA'], texts['M1']

# visualize and save
igplot = ciCOH_visual_save(ciCOH_mild_subchns, 
                           chn_area = chnInf_brainarea, 
                           chn_coord = chnInf_coord,
                           str_recchnishow = str_recchnishow,
                           lowweight = lowweight,  
                           tobesavedfile = savefile, 
                           texts = texts)

igplot.show()

print("Figure saved to " + savefile)

# del 
del ciCOH_mild_subchns, savefile, texts, title

In [ ]:
print(len(str_recchnishow))